In [99]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


## Area data has been collected and stored in below csv file

Read csv data and get dataframe

In [27]:
df = pd.read_csv('Kolkata_Neighborhood_Data.csv')
df.head()

,AREA_NAME
0,"Alipore, Kolkata, India"
1,"Baguiati, Kolkata, India"
2,"Baishnabghata Patuli Township, Kolkata, India"
3,"Ballygunge, Kolkata, India"
4,"Bara Bazar, Kolkata, India"


### Function to read location data by Geolocator

In [29]:
def Get_Latitute_Longitude(address):
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address,  timeout=None)
    latitude = 0
    longitude = 0
    if location is not None:
        latitude = location.latitude
        longitude = location.longitude
            
    xy = {'Area':address, 'Latitude':latitude, 'Longitude':longitude}
    return xy

In [30]:
# define the dataframe columns
column_names = ['Area', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [31]:
for location in df['AREA_NAME']:
    xy = Get_Latitute_Longitude(location)
    neighborhoods = neighborhoods.append(xy,ignore_index=True)

,Area,Latitude,Longitude
0,"Alipore, Kolkata, India",22.531243,88.331398
1,"Baguiati, Kolkata, India",22.610218,88.427972
2,"Baishnabghata Patuli Township, Kolkata, India",0.000000,0.000000
3,"Ballygunge, Kolkata, India",22.519162,88.372192
4,"Bara Bazar, Kolkata, India",22.580179,88.352810


In [35]:
neighborhoods = neighborhoods[neighborhoods.Latitude != 0]
neighborhoods.shape

(44, 3)

In [43]:
neighborhoods.to_csv('Kolkata_Location_Data.csv')

In [101]:
address = 'Kolkata, India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address, timeout = None)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 22.5677459, 88.3476023.


### Create mape of neiborhoods of city kolata

In [102]:
# create map of Bogotá using latitude and longitude values
map_kolkata = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, local in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Area']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kolkata)  
    
map_kolkata

#### Set foursquare credentials

In [103]:
CLIENT_ID = 'F3GUQBLLXE1PXIFIOTBAL0E2POSVO1UI1NJASFCSHGHKO5EG' # your Foursquare ID
CLIENT_SECRET = 'RSZ3JE5B5LNZGMLY4JEMIA34S51QSDS0OVKGRAI1IIMR2WKJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: F3GUQBLLXE1PXIFIOTBAL0E2POSVO1UI1NJASFCSHGHKO5EG
CLIENT_SECRET:RSZ3JE5B5LNZGMLY4JEMIA34S51QSDS0OVKGRAI1IIMR2WKJ


#### Function to get near by venues to a specific location

In [104]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [105]:
kolkata_venues = getNearbyVenues(names=neighborhoods['Area'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Alipore, Kolkata, India
Baguiati, Kolkata, India
Ballygunge, Kolkata, India
Bara Bazar, Kolkata, India
Behala, Kolkata, India
Beleghata, Kolkata, India
Belgachia, Kolkata, India
Belgharia, Kolkata, India
Bhowanipore, Kolkata, India
Bidhannagar, Kolkata, India
Birati, Kolkata, India
Bowbazar, Kolkata, India
Chitpur, Kolkata, India
Cossipore, Kolkata, India
Dhakuria, Kolkata, India
Dum Dum, Kolkata, India
Eden Gardens, Kolkata, India
Fort William, Kolkata, India
Garia, Kolkata, India
Garia railway station, Kolkata, India
Gariahat Road, Kolkata, India
Jadavpur, Kolkata, India
Joka, Kolkata, Kolkata, India
Jorasanko, Kolkata, India
Kalighat, Kolkata, India
Kalighat metro station, Kolkata, India
Kankurgachi, Kolkata, India
Kasba, Kolkata, Kolkata, India
Kavi Nazrul metro station, Kolkata, India
Mahanayak Uttam Kumar metro station, Kolkata, India
New Alipore, Kolkata, India
New Garia, Kolkata, India
North Dumdum, Kolkata, India
Park Circus, Kolkata, India
Rabindra Sarobar metro station, Kolk

In [106]:
print(kolkata_venues.shape)
kolkata_venues.head()

(3451, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Alipore, Kolkata, India",22.531243,88.331398,Taj Bengal,22.537459,88.334066,Hotel
1,"Alipore, Kolkata, India",22.531243,88.331398,India Restaurant & Caterer,22.538904,88.322278,Awadhi Restaurant
2,"Alipore, Kolkata, India",22.531243,88.331398,Victoria Memorial,22.545844,88.342890,History Museum
3,"Alipore, Kolkata, India",22.531243,88.331398,Chinoiserie,22.537733,88.334327,Chinese Restaurant
4,"Alipore, Kolkata, India",22.531243,88.331398,Balaram Mullick & Radharaman Mullick,22.533097,88.347082,Indian Sweet Shop


In [107]:
kolkata_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Alipore, Kolkata, India",100,100,100,100,100,100
"Baguiati, Kolkata, India",98,98,98,98,98,98
"Ballygunge, Kolkata, India",100,100,100,100,100,100
"Bara Bazar, Kolkata, India",100,100,100,100,100,100
"Behala, Kolkata, India",59,59,59,59,59,59
"Beleghata, Kolkata, India",100,100,100,100,100,100
"Belgachia, Kolkata, India",85,85,85,85,85,85
"Belgharia, Kolkata, India",100,100,100,100,100,100
"Bhowanipore, Kolkata, India",100,100,100,100,100,100


In [108]:
print('There are {} uniques categories.'.format(len(kolkata_venues['Venue Category'].unique())))

There are 89 uniques categories.


In [109]:
# one hot encoding
kolkata_onehot = pd.get_dummies(kolkata_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kolkata_onehot['Neighborhood'] = kolkata_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kolkata_onehot.columns[-1]] + list(kolkata_onehot.columns[:-1])
kolkata_onehot = kolkata_onehot[fixed_columns]

kolkata_onehot.head()

,Women's Store,ATM,Airport,American Restaurant,Art Museum,Arts & Crafts Store,Asian Restaurant,Awadhi Restaurant,BBQ Joint,Bakery,...,Stadium,Supermarket,Swiss Restaurant,Tea Room,Tex-Mex Restaurant,Theme Park,Tibetan Restaurant,Train Station,Tram Station,Vegetarian / Vegan Restaurant
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [110]:
kolkata_onehot.shape

(3451, 89)

#### Get kolkata grouped data where categories are columns

In [111]:
kolkata_grouped = kolkata_onehot.groupby('Neighborhood').sum().reset_index()

kolkata_grouped = kolkata_grouped[['Neighborhood','Chinese Restaurant','Park','Train Station','Bus Station','Multiplex','Indie Movie Theater','Shopping Mall']]
kolkata_grouped.head()

,Neighborhood,Chinese Restaurant,Park,Train Station,Bus Station,Multiplex,Indie Movie Theater,Shopping Mall
0,"Alipore, Kolkata, India",6,2,0,0,2,0,2
1,"Baguiati, Kolkata, India",3,1,2,1,8,0,4
2,"Ballygunge, Kolkata, India",9,2,0,0,3,0,3
3,"Bara Bazar, Kolkata, India",1,3,0,0,4,1,3
4,"Behala, Kolkata, India",3,2,0,0,2,0,1


#### Process data to determine best location

In [116]:
#Add Train Statio and Bus Station
kolkata_grouped['Station'] = kolkata_grouped['Train Station'] + kolkata_grouped['Bus Station']

#Add Multiplex and Indie Movie Theater
kolkata_grouped['Cinema'] = kolkata_grouped['Multiplex'] + kolkata_grouped['Indie Movie Theater']

#Add total column
kolkata_grouped['total'] = - 0.5 * kolkata_grouped['Chinese Restaurant'] + kolkata_grouped['Shopping Mall'] + kolkata_grouped['Station'] + kolkata_grouped['Cinema']

#Delete uneccessary columns
del kolkata_grouped['Train Station']
del kolkata_grouped['Bus Station']
del kolkata_grouped['Multiplex']
del kolkata_grouped['Indie Movie Theater']

kolkata_grouped.head()

,Neighborhood,Chinese Restaurant,Park,Shopping Mall,Station,Cinema,total
0,"Alipore, Kolkata, India",6,2,2,0,2,1.0
1,"Baguiati, Kolkata, India",3,1,4,3,8,13.5
2,"Ballygunge, Kolkata, India",9,2,3,0,3,1.5
3,"Bara Bazar, Kolkata, India",1,3,3,0,5,7.5
4,"Behala, Kolkata, India",3,2,1,0,2,1.5


#### Order by total column to order total score descending order

In [120]:
final_df = kolkata_grouped.sort_values(by='total', ascending=False).reset_index()
final_df.head()

,index,Neighborhood,Chinese Restaurant,Park,Shopping Mall,Station,Cinema,total
0,1,"Baguiati, Kolkata, India",3,1,4,3,8,13.5
1,41,"Tala, Kolkata, Kolkata, India",2,1,3,3,7,12.0
2,40,"South Dumdum, Kolkata, India",2,1,3,4,6,12.0
3,39,"Shyambazar, Kolkata, India",2,2,3,2,7,11.0
4,6,"Belgachia, Kolkata, India",2,1,3,2,7,11.0


In [121]:
print('{} is the best location to open a chinese restaurent in Kolkata'.format(final_df['Neighborhood'][0]))

Baguiati, Kolkata, India is the best location to open a chinese restaurent in Kolkata


### Baguiati, Kolkata, India is the best location to open a chinese restaurent in Kolkata